# Exploración del Dataset: EXPERIENCIAS LABORALES

Importación de Librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

# Configuración
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 50)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("✅ Librerías importadas correctamente")

Carga del Dataset

In [ ]:
# Definir rutas de forma robusta
import os
from pathlib import Path

# Obtener la ubicación del notebook
notebook_dir = Path.cwd()
print(f"Directorio actual: {notebook_dir}")

# Construir ruta al directorio data/raw
if 'notebooks' in str(notebook_dir):
    DATA_RAW_PATH = Path('../../data/raw')
    DICT_PATH = Path('../../data/diccionarios')
else:
    DATA_RAW_PATH = Path(r'E:\MTPE\data\raw')
    DICT_PATH = Path(r'E:\MTPE\data\diccionarios')

print(f"Ruta DATA_RAW_PATH: {DATA_RAW_PATH.resolve()}")
print(f"Directorio existe: {DATA_RAW_PATH.exists()}")

# Cargar dataset
archivo_csv = DATA_RAW_PATH / 'Dataset_EXPERIENCIASLABORALES.csv'
print(f"Archivo CSV: {archivo_csv.name}")
print(f"Archivo existe: {archivo_csv.exists()}")

if archivo_csv.exists():
    df_experiencias = pd.read_csv(archivo_csv, encoding='latin1', on_bad_lines='skip')
    print(f"\nDataset cargado exitosamente")
    print(f"Dimensiones: {df_experiencias.shape[0]:,} filas x {df_experiencias.shape[1]} columnas")
else:
    print(f"\nERROR: No se encuentra el archivo en: {archivo_csv.resolve()}")
    print(f"   Por favor, verifica la ruta del archivo.")

Vista Previa - Primeras Filas

In [ ]:
print(" Primeras 15 filas del dataset:")
display(df_experiencias.head(15))

Vista Previa - Últimas Filas

In [ ]:
print(" Últimas 10 filas del dataset:")
display(df_experiencias.tail(10))

Vista Previa - Muestra Aleatoria

In [ ]:
print(" Muestra aleatoria de 20 registros:")
display(df_experiencias.sample(min(20, len(df_experiencias)), random_state=42))

Información General del Dataset

In [ ]:
print(" Información del Dataset:")
print("=" * 80)
df_experiencias.info()

print("\n Columnas del Dataset:")
print("=" * 80)
for idx, col in enumerate(df_experiencias.columns, 1):
    print(f"{idx:2d}. {col} ({df_experiencias[col].dtype})")

Análisis de Valores Nulos

In [ ]:
print(" Análisis de Valores Nulos:")
print("=" * 80)

null_data = pd.DataFrame({
    'Columna': df_experiencias.columns,
    'Nulos': df_experiencias.isnull().sum(),
    '% Nulos': (df_experiencias.isnull().sum() / len(df_experiencias) * 100).round(2)
}).sort_values(by='Nulos', ascending=False)

display(null_data)

if null_data['Nulos'].sum() > 0:
    plt.figure(figsize=(12, 6))
    null_cols = null_data[null_data['Nulos'] > 0]
    plt.barh(null_cols['Columna'], null_cols['% Nulos'])
    plt.xlabel('Porcentaje de Valores Nulos (%)')
    plt.title('Valores Nulos por Columna - EXPERIENCIAS LABORALES')
    plt.tight_layout()
    plt.show()
else:
    print(" No hay valores nulos en el dataset")

Análisis de Duplicados

In [ ]:
print(" Análisis de Duplicados:")
print("=" * 80)

duplicados_totales = df_experiencias.duplicated().sum()
print(f"Registros duplicados (completos): {duplicados_totales:,} ({duplicados_totales/len(df_experiencias)*100:.2f}%)")

# Buscar columna de ID
id_cols = [col for col in df_experiencias.columns if 'ID' in col.upper()]
if id_cols:
    for id_col in id_cols:
        duplicados_id = df_experiencias[id_col].duplicated().sum()
        print(f"Duplicados en {id_col}: {duplicados_id:,} ({duplicados_id/len(df_experiencias)*100:.2f}%)")

        if duplicados_id > 0 and duplicados_id < 100:
            print(f"\n Ejemplos de {id_col} duplicados (es normal - un postulante puede tener múltiples experiencias):")
            ids_duplicados = df_experiencias[df_experiencias[id_col].duplicated(keep=False)][id_col].value_counts().head(10)
            display(ids_duplicados)

Estadísticas Descriptivas - Variables Numéricas

In [ ]:
print(" Estadísticas Descriptivas - Variables Numéricas:")
print("=" * 80)
display(df_experiencias.describe())

Estadísticas Descriptivas - Variables Categóricas

In [ ]:
print("Estadísticas Descriptivas - Variables Categóricas:")
print("=" * 80)
display(df_experiencias.describe(include=['object']))

Análisis de Cardinalidad

In [ ]:
print(" Análisis de Cardinalidad:")
print("=" * 80)

# Calcular valores únicos para cada columna
valores_unicos = [df_experiencias[col].nunique() for col in df_experiencias.columns]
tipos_dato = [str(df_experiencias[col].dtype) for col in df_experiencias.columns]
porcentaje_cardinalidad = [(df_experiencias[col].nunique() / len(df_experiencias) * 100) for col in df_experiencias.columns]

cardinalidad = pd.DataFrame({
    'Columna': df_experiencias.columns,
    'Valores_Únicos': valores_unicos,
    'Tipo_Dato': tipos_dato,
    '% Cardinalidad': [round(p, 2) for p in porcentaje_cardinalidad]
}).sort_values(by='Valores_Únicos', ascending=False)

display(cardinalidad)

Distribución de Variables Categóricas - Top 20

In [ ]:
print(" Distribución de Variables Categóricas (Top 20 valores):")
print("=" * 80)

categorical_cols = df_experiencias.select_dtypes(include=['object']).columns

for col in categorical_cols:
    print(f"\n{'='*80}")
    print(f"Columna: {col}")
    print(f"{'='*80}")
    value_counts = df_experiencias[col].value_counts()
    print(f"Valores únicos: {df_experiencias[col].nunique():,}")
    print(f"\nTop 20 valores más frecuentes:")
    display(pd.DataFrame({
        'Valor': value_counts.head(20).index,
        'Frecuencia': value_counts.head(20).values,
        'Porcentaje': (value_counts.head(20).values / len(df_experiencias) * 100).round(2)
    }))

Análisis de Empresas/Organizaciones

In [ ]:
# Buscar columna de empresa
empresa_cols = [col for col in df_experiencias.columns if 'EMPRESA' in col.upper() or 'ORGANIZACION' in col.upper() or 'EMPLEADOR' in col.upper()]

if empresa_cols:
    col_empresa = empresa_cols[0]
    print(f" Análisis de {col_empresa}:")
    print("=" * 80)

    empresa_counts = df_experiencias[col_empresa].value_counts()

    print(f"\nTotal de empresas únicas: {df_experiencias[col_empresa].nunique():,}")
    print(f"\nTop 25 empresas/organizaciones:")

    display(pd.DataFrame({
        'Empresa': empresa_counts.head(25).index,
        'Cantidad': empresa_counts.head(25).values,
        'Porcentaje': (empresa_counts.head(25).values / len(df_experiencias) * 100).round(2)
    }))

    # Visualización
    plt.figure(figsize=(14, 8))
    top_empresas = empresa_counts.head(15)
    plt.barh(range(len(top_empresas)), top_empresas.values)
    plt.yticks(range(len(top_empresas)), [str(x)[:45] for x in top_empresas.index])
    plt.xlabel('Cantidad de Registros')
    plt.title('Top 15 Empresas/Organizaciones')
    plt.tight_layout()
    plt.show()
else:
    print("No se encontró columna de empresa/organización")

Análisis de Descripción de Experiencia Laboral

In [ ]:
print(" Análisis de DESCRIPCION (Descripción de funciones/cargo):")
print("=" * 80)

if 'DESCRIPCION' in df_experiencias.columns:
    descripcion_counts = df_experiencias['DESCRIPCION'].value_counts()

    print(f"\nDescripciones únicas: {df_experiencias['DESCRIPCION'].nunique():,}")
    print(f"Valores nulos: {df_experiencias['DESCRIPCION'].isna().sum():,} ({df_experiencias['DESCRIPCION'].isna().sum()/len(df_experiencias)*100:.1f}%)")
    print(f"\nTop 30 descripciones más frecuentes:")

    display(pd.DataFrame({
        'Descripción': [str(x)[:80] for x in descripcion_counts.head(30).index],
        'Cantidad': descripcion_counts.head(30).values,
        'Porcentaje': (descripcion_counts.head(30).values / len(df_experiencias) * 100).round(2)
    }))

    # Visualización
    plt.figure(figsize=(14, 10))
    top_descripciones = descripcion_counts.head(15)
    plt.barh(range(len(top_descripciones)), top_descripciones.values)
    plt.yticks(range(len(top_descripciones)), [str(x)[:60] + '...' if len(str(x)) > 60 else str(x) for x in top_descripciones.index])
    plt.xlabel('Cantidad de Registros')
    plt.title('Top 15 Descripciones de Experiencia Laboral')
    plt.tight_layout()
    plt.show()

    # Análisis de longitud de descripciones
    df_experiencias['len_descripcion'] = df_experiencias['DESCRIPCION'].fillna('').astype(str).str.len()

    print(f"\n Estadísticas de longitud de descripciones:")
    print(f"   - Longitud mínima: {df_experiencias['len_descripcion'].min()}")
    print(f"   - Longitud máxima: {df_experiencias['len_descripcion'].max()}")
    print(f"   - Longitud promedio: {df_experiencias['len_descripcion'].mean():.1f} caracteres")
    print(f"   - Longitud mediana: {df_experiencias['len_descripcion'].median():.0f} caracteres")

    # Histograma de longitud
    plt.figure(figsize=(12, 5))
    plt.hist(df_experiencias['len_descripcion'], bins=50, edgecolor='black')
    plt.xlabel('Longitud de Descripción (caracteres)')
    plt.ylabel('Frecuencia')
    plt.title('Distribución de Longitud de Descripciones')
    plt.tight_layout()
    plt.show()
else:
    print(" No se encontró la columna DESCRIPCION")

Análisis de Fechas y Duración de Experiencia

In [ ]:
print(" Análisis de Fechas de Experiencia:")
print("=" * 80)

if 'FECHAINICIO' in df_experiencias.columns and 'FECHAFIN' in df_experiencias.columns:
    for col_fecha in ['FECHAINICIO', 'FECHAFIN']:
        print(f"\n{col_fecha}:")
        print(f"  - Valores no nulos: {df_experiencias[col_fecha].notna().sum():,}")
        print(f"  - Valores nulos: {df_experiencias[col_fecha].isna().sum():,} ({df_experiencias[col_fecha].isna().sum()/len(df_experiencias)*100:.1f}%)")

        # Mostrar ejemplos de valores
        df_temp = df_experiencias[col_fecha].dropna()
        if len(df_temp) > 0:
            print(f"  - Ejemplos: {df_temp.head(10).tolist()}")
            print(f"  - Valores únicos: {df_experiencias[col_fecha].nunique():,}")

    # Intentar convertir a datetime para análisis temporal
    print("\n" + "="*80)
    print(" Intentando parsear fechas...")

    try:
        # Probar diferentes formatos
        df_experiencias['FECHAINICIO_parsed'] = pd.to_datetime(df_experiencias['FECHAINICIO'], errors='coerce', format='mixed')
        df_experiencias['FECHAFIN_parsed'] = pd.to_datetime(df_experiencias['FECHAFIN'], errors='coerce', format='mixed')

        fechas_inicio_validas = df_experiencias['FECHAINICIO_parsed'].notna().sum()
        fechas_fin_validas = df_experiencias['FECHAFIN_parsed'].notna().sum()

        print(f" Fechas de inicio parseadas: {fechas_inicio_validas:,} ({fechas_inicio_validas/len(df_experiencias)*100:.1f}%)")
        print(f" Fechas de fin parseadas: {fechas_fin_validas:,} ({fechas_fin_validas/len(df_experiencias)*100:.1f}%)")

        if fechas_inicio_validas > 0:
            print(f"\n Rango de fechas de inicio:")
            print(f"   - Fecha mínima: {df_experiencias['FECHAINICIO_parsed'].min()}")
            print(f"   - Fecha máxima: {df_experiencias['FECHAINICIO_parsed'].max()}")

        if fechas_fin_validas > 0:
            print(f"\n Rango de fechas de fin:")
            print(f"   - Fecha mínima: {df_experiencias['FECHAFIN_parsed'].min()}")
            print(f"   - Fecha máxima: {df_experiencias['FECHAFIN_parsed'].max()}")

        # Calcular duración en días
        if fechas_inicio_validas > 0 and fechas_fin_validas > 0:
            df_experiencias['duracion_dias'] = (df_experiencias['FECHAFIN_parsed'] - df_experiencias['FECHAINICIO_parsed']).dt.days
            df_experiencias['duracion_meses'] = (df_experiencias['duracion_dias'] / 30.44).round(1)
            df_experiencias['duracion_anos'] = (df_experiencias['duracion_dias'] / 365.25).round(2)

            # Filtrar duraciones válidas (positivas)
            duraciones_validas = df_experiencias[df_experiencias['duracion_dias'] > 0]

            if len(duraciones_validas) > 0:
                print(f"\n Análisis de Duración de Experiencia:")
                print(f"   - Registros con duración calculable: {len(duraciones_validas):,}")
                print(f"   - Duración mínima: {duraciones_validas['duracion_dias'].min()} días ({duraciones_validas['duracion_meses'].min()} meses)")
                print(f"   - Duración máxima: {duraciones_validas['duracion_dias'].max()} días ({duraciones_validas['duracion_anos'].max():.1f} años)")
                print(f"   - Duración promedio: {duraciones_validas['duracion_dias'].mean():.0f} días ({duraciones_validas['duracion_meses'].mean():.1f} meses)")
                print(f"   - Duración mediana: {duraciones_validas['duracion_dias'].median():.0f} días ({duraciones_validas['duracion_meses'].median():.1f} meses)")

                # Visualización de duración en meses
                fig, axes = plt.subplots(1, 2, figsize=(15, 5))

                # Histograma (limitado a 60 meses para mejor visualización)
                duraciones_plot = duraciones_validas[duraciones_validas['duracion_meses'] <= 60]['duracion_meses']
                axes[0].hist(duraciones_plot, bins=50, edgecolor='black')
                axes[0].set_xlabel('Duración (meses)')
                axes[0].set_ylabel('Frecuencia')
                axes[0].set_title('Distribución de Duración de Experiencia (≤60 meses)')
                axes[0].axvline(duraciones_validas['duracion_meses'].median(), color='red', linestyle='--', label=f'Mediana: {duraciones_validas["duracion_meses"].median():.1f} meses')
                axes[0].legend()

                # Boxplot
                axes[1].boxplot(duraciones_validas[duraciones_validas['duracion_meses'] <= 60]['duracion_meses'])
                axes[1].set_ylabel('Duración (meses)')
                axes[1].set_title('Boxplot de Duración de Experiencia (≤60 meses)')

                plt.tight_layout()
                plt.show()

                # Distribución por rangos de duración
                print(f"\n Distribución por rangos de duración:")
                bins = [0, 3, 6, 12, 24, 36, 60, 120, float('inf')]
                labels = ['0-3 meses', '3-6 meses', '6-12 meses', '1-2 años', '2-3 años', '3-5 años', '5-10 años', '>10 años']
                duraciones_validas['rango_duracion'] = pd.cut(duraciones_validas['duracion_meses'], bins=bins, labels=labels)

                rango_counts = duraciones_validas['rango_duracion'].value_counts().sort_index()
                display(pd.DataFrame({
                    'Rango': rango_counts.index,
                    'Cantidad': rango_counts.values,
                    'Porcentaje': (rango_counts.values / len(duraciones_validas) * 100).round(2)
                }))

                plt.figure(figsize=(12, 6))
                plt.bar(range(len(rango_counts)), rango_counts.values)
                plt.xticks(range(len(rango_counts)), rango_counts.index, rotation=45, ha='right')
                plt.xlabel('Rango de Duración')
                plt.ylabel('Cantidad')
                plt.title('Distribución de Experiencias Laborales por Duración')
                plt.tight_layout()
                plt.show()
            else:
                print("\n⚠️ No se encontraron duraciones válidas (positivas)")

    except Exception as e:
        print(f" Error al procesar fechas: {str(e)}")
        print("   Las fechas pueden tener un formato no estándar que requiere limpieza")

else:
    print(" No se encontraron columnas FECHAINICIO y/o FECHAFIN")

Análisis de Postulantes y Experiencias

In [ ]:
# Buscar columna de ID de postulante
id_post_cols = [col for col in df_experiencias.columns if 'POST' in col.upper() and 'ID' in col.upper()]

if id_post_cols:
    col_id_post = id_post_cols[0]
    print(f" Análisis de {col_id_post}:")
    print("=" * 80)

    postulantes_unicos = df_experiencias[col_id_post].nunique()
    total_registros = len(df_experiencias)

    print(f"Total de postulantes únicos: {postulantes_unicos:,}")
    print(f"Total de registros: {total_registros:,}")
    print(f"Promedio de experiencias por postulante: {total_registros/postulantes_unicos:.2f}")

    # Distribución de experiencias por postulante
    exp_por_persona = df_experiencias[col_id_post].value_counts()

    print(f"\n Distribución de número de experiencias por postulante:")
    dist_exp = exp_por_persona.value_counts().sort_index()

    # Mostrar primeros valores
    display(pd.DataFrame({
        'Num_Experiencias': dist_exp.head(25).index,
        'Cantidad_Postulantes': dist_exp.head(25).values,
        'Porcentaje': (dist_exp.head(25).values / postulantes_unicos * 100).round(2)
    }))

    # Visualización
    plt.figure(figsize=(12, 6))
    top_dist = dist_exp.head(15)
    plt.bar(top_dist.index.astype(str), top_dist.values)
    plt.xlabel('Número de Experiencias Laborales')
    plt.ylabel('Cantidad de Postulantes')
    plt.title('Distribución de Número de Experiencias por Postulante (Top 15)')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # Postulantes con más experiencias
    print(f"\n Top 10 postulantes con más experiencias:")
    top_exp = exp_por_persona.head(10)
    for idx, (id_post, cantidad) in enumerate(top_exp.items(), 1):
        print(f"   {idx}. ID: {str(id_post)[:30]}... - {cantidad} experiencias")
else:
    print(" No se encontró columna de ID de postulante")

Análisis de Rango Salarial

In [ ]:
print(" Análisis de RANGO_SALARIAL:")
print("=" * 80)

if 'RANGO_SALARIAL' in df_experiencias.columns:
    rango_counts = df_experiencias['RANGO_SALARIAL'].value_counts()

    print(f"\nRangos salariales únicos: {df_experiencias['RANGO_SALARIAL'].nunique():,}")
    print(f"Valores nulos: {df_experiencias['RANGO_SALARIAL'].isna().sum():,} ({df_experiencias['RANGO_SALARIAL'].isna().sum()/len(df_experiencias)*100:.1f}%)")
    print(f"\nDistribución de rangos salariales:")

    display(pd.DataFrame({
        'Rango Salarial': rango_counts.index,
        'Cantidad': rango_counts.values,
        'Porcentaje': (rango_counts.values / len(df_experiencias) * 100).round(2)
    }))

    # Visualización
    plt.figure(figsize=(12, 8))
    plt.barh(range(len(rango_counts)), rango_counts.values)
    plt.yticks(range(len(rango_counts)), [str(x) for x in rango_counts.index])
    plt.xlabel('Cantidad')
    plt.title('Distribución de Rangos Salariales')
    plt.tight_layout()
    plt.show()

else:
    print(" No se encontró la columna RANGO_SALARIAL")